In [1]:
import yaml

from jinja2 import Environment, FileSystemLoader

from create_network import *
from model_merging.task_vectors import MTLTaskVector
from model_merging.utils import aggregate_task_vectors
from model_merging.eval_utils import perform_eval_with_merged_vector
from utils import *

In [2]:
env = Environment(loader=FileSystemLoader('.'))
template = env.get_template('config/mtl.yaml.j2')
rendered_yaml = template.render()
mm_config = yaml.safe_load(rendered_yaml)

model_classes = {
  "split": MTLDeepLabv3,
  "mtan": MTANDeepLabv3
}

In [3]:
pt_model = torch_load(mm_config["model_merging"]["pt_model_file"])

In [4]:
pt_model = torch_load(mm_config["model_merging"]["pt_model_file"])
task_vectors = [MTLTaskVector(pt_model, ft_file) for ft_file in mm_config["model_merging"]["ft_model_files"]]

In [ ]:
mtl_task_vector = aggregate_task_vectors(task_vectors, mm_config)

Create pre-trained model with task-specific heads

In [ ]:
train_tasks_str = ' + '.join(task.title() for task in mtl_task_vector.tasks.keys())
print(f"Dataset: {mm_config['model_merging']['dataset'].title()} | Training Task: {train_tasks_str}")

# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pt_model = model_classes[mm_config["model_merging"]["network"]](mtl_task_vector.tasks)
# ft_models = {
#     next(iter(ft_model.tasks.keys())): ft_model
#     for ft_file in mm_config["model_merging"]["ft_model_files"]
#     for ft_model in [torch_load(ft_file)]
# }

In [7]:
# aa =pt_model.state_dict()["shared_conv.0.weight"] + 1 * mtl_task_vector.vector["shared_conv.0.weight"]
# aa

Evaluate model

In [ ]:
pt_model.tasks

In [ ]:
mtl_task_vector.tasks

In [ ]:
perform_eval_with_merged_vector(pt_model, mtl_task_vector, mm_config)

In [ ]:
# torch_save(model, 'logging/pt_model.pt')
# torch_load('logging/model_test.pt')